In [1]:
# Cell 1: Install dependencies (run once if needed)
# - Installs pdfplumber, pandas and yfinance for extraction & verification.
!pip install pdfplumber pandas yfinance python-dateutil --quiet


In [2]:
# Cell 2: Imports and typing
# - Imports required libraries and typing helpers for later cells.
import pdfplumber
import pandas as pd
import re
import json
from typing import List, Tuple, Dict, Any, Optional
from datetime import datetime
import yfinance as yf


In [3]:
# Cell 3: PDF cleaning helper
# - Normalizes whitespace and removes heavily numeric table lines to reduce noise.
def pdf_clean(text: str) -> str:
    t = re.sub(r'\u00A0', ' ', text)                      # NBSP
    t = re.sub(r'[ \t]+', ' ', t)                         # collapse spaces/tabs
    lines = []
    for line in t.splitlines():
        line = line.strip()
        if not line:
            continue
        toks = line.split()
        numy = sum(1 for w in toks if re.fullmatch(r'[\$€£₹]?[-()]?\d[\d,\.]*%?', w))
        if numy / max(1, len(toks)) > 0.6:
            continue
        lines.append(line)
    cleaned = "\n".join(lines)
    cleaned = re.sub(r'\n{2,}', '\n\n', cleaned)
    return cleaned.strip()


In [4]:
# Cell 4: Extract text and page-level tables using pdfplumber
# - Returns full_text string and list of (page_number, pandas.DataFrame) for any tables detected.
def extract_text_and_tables_from_pdf(path_or_file) -> Tuple[str, List[Tuple[int, pd.DataFrame]]]:
    text_pages = []
    tables = []
    # path_or_file can be a file path or a file-like object (e.g., uploaded file)
    with pdfplumber.open(path_or_file) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            txt = page.extract_text() or ""
            text_pages.append(txt)
            try:
                for tbl in page.extract_tables():
                    if not tbl:
                        continue
                    header = tbl[0]
                    rows = tbl[1:] if len(tbl) > 1 else []
                    # ensure columns are unique strings
                    cols = [str(h) if h and str(h).strip() else f"col_{j}" for j,h in enumerate(header)]
                    df = pd.DataFrame(rows, columns=cols)
                    tables.append((i, df))
            except Exception:
                pass
    full_text = "\n\n".join(text_pages)
    return full_text, tables


In [5]:
# Cell 5: Section segmentation helpers
# - Splits cleaned text into paragraphs and identifies sections by keywords.
SECTION_KEYWORDS = {
    "Executive Summary": ["executive summary", "overview", "summary of operations", "company overview"],
    "MD&A": ["management's discussion", "management discussion and analysis", "md&a"],
    "Financial Statements": ["financial statements", "consolidated statements", "consolidated balance sheet"],
    "Risk Factors": ["risk factors", "risks related to"],
    "Notes": ["notes to the consolidated", "notes to financial statements"],
}

def split_paragraphs(text: str) -> List[str]:
    return [p.strip() for p in re.split(r'\n{2,}', text) if p.strip()]

def segment_sections(text: str) -> Dict[str, str]:
    paragraphs = split_paragraphs(text)
     # quick heading detection: build index set of paragraph indices that look like headings
    heading_idxs = set()
    for i, p in enumerate(paragraphs):
        stripped = p.strip()
        # uppercase short heading or typical SEC "Item X." markers
        if (len(stripped.split()) <= 8 and stripped.isupper()) or re.search(r'^\s*Item\s+\d+[\.\:]', stripped, re.I):
            heading_idxs.add(i)

    sections = {k: "" for k in SECTION_KEYWORDS.keys()}
    for idx, p in enumerate(paragraphs):
        low = p.lower()
        for sect, keys in SECTION_KEYWORDS.items():
            if any(k in low for k in keys):
                buf = [p]
                j = idx + 1
                while j < len(paragraphs) and j not in heading_idxs and len(paragraphs[j].split()) > 6:
                    buf.append(paragraphs[j])
                    j += 1
                sections[sect] = "\n\n".join(buf)
                break
    if not sections["Executive Summary"] and paragraphs:
        sections["Executive Summary"] = "\n\n".join(paragraphs[:2])
    return sections


In [6]:
# Cell 6 (replacement): Robust table normalization and numeric cleaning
# - Uses .iloc to avoid Series key/position ambiguity and DataFrame.apply + Series.map
# - Returns numeric DataFrame (same shape) with None where parsing failed.

from typing import Any, Optional
import pandas as pd
import re

def clean_cell_value(v: Any) -> Optional[float]:
    if v is None:
        return None
    s = str(v).strip()
    if s in {"", "-", "—", "na", "n/a"}:
        return None
    s = re.sub(r'\[\d+\]','', s)   # remove footnote markers like [1]
    if '%' in s:
        try:
            return float(s.replace('%','').replace(',',''))/100.0
        except:
            return None
    neg = False
    if s.startswith('(') and s.endswith(')'):
        neg = True
        s = s[1:-1]
    s = s.replace('$','').replace(',','').replace('—','').strip()
    try:
        val = float(s)
        return -val if neg else val
    except:
        return None

def normalize_table_dataframe(df: pd.DataFrame, context_text: str = "") -> pd.DataFrame:
    """
    Convert a raw table DataFrame to numeric where possible.
    Uses context_text to detect scale (in millions/thousands).
    """
    # copy as strings to avoid modifying original
    numeric = df.copy(deep=True)

    # detect scale from context
    ctx = (context_text or "").lower()
    scale = 1.0
    if 'in millions' in ctx or 'amounts in millions' in ctx:
        scale = 1e6
    elif 'in thousands' in ctx or 'amounts in thousands' in ctx:
        scale = 1e3
    elif 'in billions' in ctx:
        scale = 1e9

    # iterate columns by position to avoid integer-label ambiguity
    for idx in range(numeric.shape[1]):
        # convert column to string first to ensure .map works
        col = numeric.iloc[:, idx].astype(str)
        # apply parser
        parsed = col.map(lambda x: clean_cell_value(x) if x and x != 'nan' else None)
        # scale numeric values
        parsed = parsed.map(lambda x: None if x is None else x * scale)
        # assign back
        numeric.iloc[:, idx] = parsed

    # coerce dtype: keep as object (mixture of None/float) for display
    return numeric


In [7]:
# Cell 7: Guess table type heuristics
# - Looks for key phrases in the table text to classify statement type.
def guess_table_type(df: pd.DataFrame) -> str:
    text = " ".join(df.astype(str).fillna('').values.flatten()).lower()
    # look at header row too
    header_text = " ".join([str(c).lower() for c in list(df.columns)])
    text_all = header_text + " " + text
    if any(k in text_all for k in ["total assets", "total liabilities", "shareholders' equity", "total equity", "assets", "liabilities"]):
        return "Balance Sheet"
    if any(k in text_all for k in ["net income", "profit", "loss", "revenue", "sales", "operating income", "cost of goods sold", "income before"]):
        return "Income Statement"
    if any(k in text_all for k in ["cash flows", "net cash", "cash and cash equivalents", "operating activities", "investing activities", "financing activities"]):
        return "Cash Flow"
    # if first column contains "year" or "period" and other columns are numeric -> likely financial
    first_col = " ".join(df.iloc[:,0].astype(str).fillna('').values).lower()
    if "period" in first_col or re.search(r'\b(20\d{2})\b', first_col):
        return "Likely Financial Table"
    return "Unknown"


In [8]:
# Cell 8: Ticker extraction & verification
# - Finds probable tickers (parentheses / explicit labels) and fetches price + name with yfinance.
def extract_probable_tickers(text: str) -> List[str]:
    found = []
    for m in re.finditer(r'\((?P<t>[A-Z]{1,5}(?:\.[A-Z])?)\)', text):
        found.append(m.group("t"))
    for m in re.finditer(r'(?:ticker|symbol)\s*[:\-]\s*([A-Z]{1,5}(?:\.[A-Z])?)', text, re.I):
        found.append(m.group(1).upper())
    if "apple" in text.lower() and "AAPL" not in found:
        found.append("AAPL")
    if "tesla" in text.lower() and "TSLA" not in found:
        found.append("TSLA")
    return list(dict.fromkeys(found))

def verify_tickers_with_yfinance(tickers: List[str]) -> List[Dict[str, Any]]:
    verified = []
    for t in tickers:
        try:
            tk = yf.Ticker(t)
            info = tk.info or {}
            price = info.get("regularMarketPrice")
            name = info.get("longName") or info.get("shortName")
            if price is not None:
                verified.append({"ticker": t, "price": price, "name": name or "N/A"})
        except Exception:
            pass
    return verified


In [9]:
# new helper: filter small/garbage tables
def filter_tables(raw_tables: List[Tuple[int, pd.DataFrame]],
                  min_rows: int = 2,
                  min_cols: int = 2,
                  min_text_ratio: float = 0.2) -> List[Tuple[int, pd.DataFrame]]:
    """
    Keep tables that meet simple heuristics:
      - at least `min_rows` rows
      - at least `min_cols` columns
      - at least `min_text_ratio` of cells contain non-numeric text (helpful for header detection)
    """
    kept = []
    for page, df in raw_tables:
        if df.shape[0] < min_rows or df.shape[1] < min_cols:
            continue
        # compute fraction of cells that look textual (non purely numeric)
        total = df.size
        texty = 0
        for cell in df.values.flatten():
            s = str(cell).strip()
            # treat empty / hyphen as non-text
            if not s or s in {"-", "—", "nan"}:
                continue
            # consider as text if contains letters or at least mixed chars
            if re.search(r'[A-Za-z]', s) or re.search(r'[,A-Za-z\$₹£€]', s):
                texty += 1
        if total == 0:
            continue
        if (texty / total) < min_text_ratio:
            # likely a numeric-only table (which might be okay) — but often it's a fragment; skip for now
            continue
        kept.append((page, df))
    return kept


In [10]:
# Cell 9: End-to-end pipeline function
# - Given path/file, returns cleaned text, detected sections, parsed tables (raw+numeric+type), and verified tickers.
def analyze_pdf_file(path_or_file) -> Dict[str, Any]:
    raw_text, tables = extract_text_and_tables_from_pdf(path_or_file)
    cleaned = pdf_clean(raw_text)
    sections = segment_sections(cleaned)
        # inside analyze_pdf_file after obtaining tables:
    # filter out tiny/garbage tables first
    tables_filtered = filter_tables(tables, min_rows=2, min_cols=2, min_text_ratio=0.15)

    parsed_tables = []
    for page, df in tables_filtered:
        # create small page-context snippet (approx)
        ctx_lines = cleaned.splitlines()
        start_line = max(0, (page-1)*10)
        context = "\n".join(ctx_lines[start_line: start_line + 20])
        numeric_df = normalize_table_dataframe(df, context_text=context)
        ttype = guess_table_type(df)
        parsed_tables.append({"page": page, "type": ttype, "raw": df, "numeric": numeric_df})

    tickers = extract_probable_tickers(cleaned)
    verified = verify_tickers_with_yfinance(tickers)
    return {"text": cleaned, "sections": sections, "tables": parsed_tables, "verified": verified}


In [11]:
# Cell 10 (replacement): Demo run — print filtered table counts and display first kept table
# - Run analyze_pdf_file on a PDF, show how many tables remained after filtering and display the first one.
from IPython.display import display
import pandas as pd

# <-- update this to your PDF file path (use raw string r"..." or forward slashes)
pdf_path = r"D:/Finance-Insight/data/raw/filings/Tesla_10-Q.pdf"

# run pipeline
res = analyze_pdf_file(pdf_path)

# basic summary
print("=== Sections detected ===")
for name, body in res["sections"].items():
    print(f"- {name}: {'FOUND' if body else 'not found'}")

tables = res.get("tables", [])
print(f"\n=== Tables found by pdfplumber (raw count) === {len(tables)}")

# If you used filter_tables inside analyze_pdf_file, parsed_tables is the filtered list.
parsed_tables = res.get("tables", [])  # analyze_pdf_file returns parsed tables (filtered + numeric)
print(f"=== Parsed / kept tables (after filtering) === {len(parsed_tables)}")

if not parsed_tables:
    print("No parsed tables kept. Try lowering filter thresholds (min_rows/min_cols/min_text_ratio) or inspect raw extraction.")
else:
    # show pages kept
    pages = [t["page"] for t in parsed_tables]
    print("Pages with kept tables:", pages)

    # display first kept table (raw + numeric)
    first = parsed_tables[0]
    print(f"\n--- First kept table: Page {first['page']} — Type: {first.get('type','Unknown')} ---\n")
    print("Raw (head):")
    display(first["raw"].head(10))
    print("Numeric (normalized) preview (head):")
    display(first["numeric"].head(10))

    # show small diagnostics for the first table
    raw_shape = first["raw"].shape
    num_shape = first["numeric"].shape
    print(f"\nRaw table shape: {raw_shape}; Numeric table shape: {num_shape}")
    # show few sample cells that didn't parse (to help tune clean_cell_value)
    sample_bad = []
    for r in range(min(5, num_shape[0])):
        for c in range(min(5, num_shape[1])):
            val = first["numeric"].iloc[r, c]
            if val is None:
                sample_bad.append((r, c, str(first["raw"].iloc[r, c])))
    if sample_bad:
        print("\nSample unparsed cells (row, col, raw):")
        for r,c,raw in sample_bad[:10]:
            print(f"  - ({r},{c}) -> {raw}")

# show verified tickers
print("\n=== Verified tickers (yfinance) ===")
verified = res.get("verified", [])
if verified:
    display(pd.DataFrame(verified))
else:
    print("No tickers verified automatically. Try adding company name or 'ticker: AAPL' in the text.")


=== Sections detected ===
- Executive Summary: FOUND
- MD&A: not found
- Financial Statements: not found
- Risk Factors: not found
- Notes: not found

=== Tables found by pdfplumber (raw count) === 35
=== Parsed / kept tables (after filtering) === 35
Pages with kept tables: [4, 5, 6, 7, 7, 8, 8, 9, 10, 11, 12, 13, 13, 14, 15, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 19, 20, 24, 25, 25, 25, 25, 29, 30, 34]

--- First kept table: Page 4 — Type: Balance Sheet ---

Raw (head):


,Assets,col_1,col_2,col_3,col_4,col_5
0,Current assets,,None,,,None
1,Cash and cash equivalents,$,"15,587",,$,"16,139"
2,Short-term investments,"21,195",None,,"20,424",None
3,"Accounts receivable, net","3,838",None,,"4,418",None
4,Inventory,"14,570",None,,"12,017",None
5,Prepaid expenses and other current assets,"5,943",None,,"5,362",None
6,Total current assets,"61,133",None,,"58,360",None
7,"Operating lease vehicles, net","5,230",None,,"5,581",None
8,"Solar energy systems, net","4,788",None,,"4,924",None
9,"Property, plant and equipment, net","38,574",None,,"35,836",None


Numeric (normalized) preview (head):


,Assets,col_1,col_2,col_3,col_4,col_5
0,None,NaN,NaN,None,NaN,NaN
1,None,NaN,15587.0,None,NaN,16139.0
2,None,21195.0,NaN,None,20424.0,NaN
3,None,3838.0,NaN,None,4418.0,NaN
4,None,14570.0,NaN,None,12017.0,NaN
5,None,5943.0,NaN,None,5362.0,NaN
6,None,61133.0,NaN,None,58360.0,NaN
7,None,5230.0,NaN,None,5581.0,NaN
8,None,4788.0,NaN,None,4924.0,NaN
9,None,38574.0,NaN,None,35836.0,NaN



Raw table shape: (40, 6); Numeric table shape: (40, 6)

Sample unparsed cells (row, col, raw):
  - (0,0) -> Current assets
  - (0,3) -> 
  - (1,0) -> Cash and cash equivalents
  - (1,3) -> 
  - (2,0) -> Short-term investments
  - (2,3) -> 
  - (3,0) -> Accounts receivable, net
  - (3,3) -> 
  - (4,0) -> Inventory
  - (4,3) -> 

=== Verified tickers (yfinance) ===


,ticker,price,name
0,TSLA,401.25,"Tesla, Inc."


In [12]:
# Cell 11: (Optional) Write a reusable module file segmentation_and_tables.py
# - This exports analyze_pdf_file and helpers so app.py can import them.
module_code = """
# segmentation_and_tables.py
# Generated from milestone4 notebook - contains helpers for segmentation & table parsing.
# (Only a wrapper that imports functions from the notebook environment if needed.)
# Note: If running as a module, ensure dependencies (pdfplumber, pandas, yfinance) are installed.
from typing import List, Tuple, Dict, Any, Optional
import re, pdfplumber, pandas as pd
from datetime import datetime
# (Paste the function definitions from notebook here when moving to a file.)
# For brevity, create this module manually by copying functions: pdf_clean, extract_text_and_tables_from_pdf, segment_sections, normalize_table_dataframe, guess_table_type, extract_probable_tickers, verify_tickers_with_yfinance, analyze_pdf_file
"""
with open("../segmentation_and_tables.py", "w", encoding="utf-8") as f:
    f.write(module_code)
print("Wrote ../segmentation_and_tables.py (edit it to paste functions for production).")


Wrote ../segmentation_and_tables.py (edit it to paste functions for production).


In [13]:
# Cell 12: Integration instructions (copy-paste)
# - To integrate with your app.py:
print("""
Integration steps:
1) Copy stable functions (pdf_clean, extract_text_and_tables_from_pdf, segment_sections,
   normalize_table_dataframe, guess_table_type, extract_probable_tickers, verify_tickers_with_yfinance, analyze_pdf_file)
   into a new file at project root named `segmentation_and_tables.py`.

2) In app.py (Streamlit), import:
   from segmentation_and_tables import analyze_pdf_file

3) When user uploads a PDF in Streamlit, pass the uploaded file object to analyze_pdf_file(uploaded_file)
   and display sections and tables: st.expander for sections, st.dataframe for tables.

4) Add a 'Download JSON' button:
   st.download_button(..., json.dumps(result, default=str), file_name="milestone4_results.json")

If you want, I can now produce the actual `segmentation_and_tables.py` file content ready to paste.
""")



Integration steps:
1) Copy stable functions (pdf_clean, extract_text_and_tables_from_pdf, segment_sections,
   normalize_table_dataframe, guess_table_type, extract_probable_tickers, verify_tickers_with_yfinance, analyze_pdf_file)
   into a new file at project root named `segmentation_and_tables.py`.

2) In app.py (Streamlit), import:
   from segmentation_and_tables import analyze_pdf_file

3) When user uploads a PDF in Streamlit, pass the uploaded file object to analyze_pdf_file(uploaded_file)
   and display sections and tables: st.expander for sections, st.dataframe for tables.

4) Add a 'Download JSON' button:
   st.download_button(..., json.dumps(result, default=str), file_name="milestone4_results.json")

If you want, I can now produce the actual `segmentation_and_tables.py` file content ready to paste.

